In [1]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

2024-01-07 10:34:20.339093: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 10:34:20.384407: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 10:34:20.385259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 10:34:21.199182: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Avoid OOM errors by setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2024-01-07 10:34:23.005812: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 10:34:23.046071: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
for gpu in gpus:
    print(gpu)

In [5]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')  # the same

In [20]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [21]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

## Collect positive and anchor classes

In [23]:
# Import uuid library to generate unique image names
import uuid

In [37]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'data/anchor/aa02651c-ad3e-11ee-b2f0-d43d7edce4fa.jpg'

In [6]:
OXFF = 0xFF

# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Cut down frame to 250x250px
    frame =  frame[120:120+250,200:200+250, :]

    # Collect anchors
    if cv2.waitKey(1) & OXFF == ord('a'):
        # Create the unique file path
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)

    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path
        imgname = os.pqqath.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)

    # Show image back to screen
    cv2.imshow('Image Collection', frame)

    # Breaking gracefull
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread (0x9269bc0).
Cannot move to target thread (0x80a6d00)

QObject::moveToThread: Current thread (0x80a6d00) is not the object's thread

AttributeError: module 'os' has no attribute 'pqqath'